In [ ]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.applications.densenet import DenseNet121

from utility import get_sample_counts
from generator import AugmentedImageSequence
from sklearn.metrics import roc_auc_score
import numpy as np

In [ ]:
# pre-trained weights from https://github.com/brucechou1983/CheXNet-Keras
weights_path = "pre-trained_weight.h5"
data_path = "data/default_split"
image_path = "data/images"

In [ ]:
class_names = ['Atelectasis',
            'Cardiomegaly',
            'Consolidation',
            'Edema',
            'Effusion',
            'Emphysema',
            'Fibrosis',
            'Hernia',
            'Infiltration',
            'Mass',
            'Nodule',
            'Pleural_Thickening',
            'Pneumonia',
            'Pneumothorax']

def get_model(weights_path=weights_path, input_shape=(224, 224, 3)):
    img_input = Input(shape=input_shape)

    base_model = DenseNet121(
        include_top=False,
        input_tensor=img_input,
        input_shape=input_shape,
        weights=None,
        pooling="avg")

    x = base_model.output
    predictions = tf.keras.layers.Dense(14, activation='sigmoid', name='predictions')(x)
    model = Model(inputs=img_input, outputs=predictions)

    if weights_path is not None:
        print(f"load model weights_path: {weights_path}")
        model.load_weights(weights_path)
    
    return model

model = get_model(class_names)           

In [ ]:
test_counts, _ = get_sample_counts(data_path, "test", class_names)


test_ds = AugmentedImageSequence(
        dataset_csv_file=data_path +"/dev.csv",
        class_names=class_names,
        source_image_dir=image_path,
        batch_size=32,
        target_size=(224, 224),
        augmenter=None,
        steps=int(test_counts / 32),
        shuffle_on_epoch_end=False,
    )

In [ ]:
y_hat = model.predict(test_ds, verbose=1)
y = test_ds.get_y_true()

In [ ]:
aurocs = []
with open("score.txt", "w") as f:
    for i in range(len(class_names)):
        try:
            score = roc_auc_score(y[:, i], y_hat[:, i])
            aurocs.append(score)
        except ValueError:
            score = 0
        f.write(f"{class_names[i]}: {score}\n")
    mean_auroc = np.mean(aurocs)
    f.write("-------------------------\n")
    f.write(f"mean auroc: {mean_auroc}\n")
    print(f"mean auroc: {mean_auroc}")

Results see test.log